In [28]:
import pandas as pd
final = pd.read_csv('final_cleaned_data_1.csv')
final  = final.iloc[:,2:]
X = final.drop(['masked_account','Paid_In_30',],axis = 1)
Y = final['Paid_In_30']

from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
X.Credit_tier = enc.fit_transform(X.Credit_tier)

enc_labels = LabelEncoder()
Y = enc_labels.fit_transform(Y)

# Selected Variables

In [44]:
X.columns

Index(['ApplicantFICOScore', 'DTI', 'LTV', 'PTI_BOOKED', 'CUSTOM_SCORE', 'MOB',
       'BACKEND_TOTAL', 'LINE4_LTV', 'F0066_CONTACT_NO_PROMISE_COUNT',
       'DSL_NSF', 'F0065_PROMISES_TAKEN_COUNT',
       'F0068_ATTEMPTED_CONTACT_COUNT', 'F0071_CORRESPONDENCE_COUNT',
       'F0283_PROMISES_KEPT_LIFE_TO_DATE',
       'F0304_TODAY_MINUS_PROMISES_TAKEN_DATE',
       'F0307_TODAY_MINUS_ATTEMPTED_CONTACT_DATE',
       'F0314_TODAY_MINUS_DAYS_LAST_CONTACT', 'F0381_DAYS_DELINQUENT_360',
       'PROMISE_STATUS_O', 'BALANCE_PERCENT_LOAN', 'DSLP',
       'EXTENSION_ELIGIBLE_Y', 'F0180_CURRENT_INTEREST_RATE',
       'F0198_NUMBER_OF_PAYMENTS_PAID', 'F0217_NSF_HISTORY',
       'F0223_NUMBER_OF_EXTENSIONS', 'PMT_RAT_OBS', 'lastchannel_Z',
       'PaymentAmtLast60Days', 'PaymentAmtLast90Days', 'PaymentNumLast120Days',
       'PmtNumFullAmtLast60Days', 'LIFETIME_COUNT_Paymentus_Debit',
       'D31P_TOT_2', 'D9P_TOT_3', 'D9P_TOT_4', 'D9P_TOT_6', 'D5P_TOT_4',
       'D5P_TOT_1', 'D31P_TOT_6', 'DD_AVG_

In [43]:
X.columns.to_series().to_csv('Selected_variables.csv',index =False)

# Preprocessing for Model

In [29]:
from tensorflow.keras.utils import to_categorical
Y_onehot = to_categorical(Y,num_classes=2)

In [30]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X),columns = X.columns)

In [31]:
from sklearn.model_selection import train_test_split

# Splitting Data into Training and Testing Data
x_train,x_val,y_train,y_val = train_test_split(X_scaled,Y_onehot,random_state =7,test_size=0.3)

# Splitting Data into Training and Testing Data
x_test,x_val,y_test,y_val = train_test_split(x_val,y_val,random_state =7,test_size=0.5)

In [12]:
x_train.shape

(32939, 56)

In [1]:
import tensorflow as tf

NN_model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(50,input_shape = (56,)), # Dropouts are generally used after Activation functions but it can be used before for relu
        tf.keras.layers.Activation('tanh'),
        tf.keras.layers.Dense(25),  # Dropouts are generally used after Activation functions but it can be used before for relu
        tf.keras.layers.Activation('tanh'),
        tf.keras.layers.Dense(10),  # Dropouts are generally used after Activation functions but it can be used before for relu
        tf.keras.layers.Activation('tanh'),       
        tf.keras.layers.Dense(2,activation = 'softmax')
    ])

In [175]:
adam = tf.keras.optimizers.Adam(learning_rate= 0.000001,beta_1 =0.9 ,beta_2=0.99,decay = 1e-4)

# Compile
NN_model.compile(optimizer= adam,loss='binary_crossentropy',metrics=[tf.keras.metrics.AUC(from_logits=True)])
# NN_model.compile(optimizer= adam,loss='binary_crossentropy',metrics=['accuracy'])

In [176]:
EarlyStopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5,min_delta = 0.00000001)

NN_model.fit(X,Y,epochs=500,validation_data=(x_val,y_val),batch_size = int(X.shape[0]/50),callbacks=EarlyStopping)

Epoch 1/500
51/51 [==============================] - 2s 15ms/step - loss: 0.2315 - auc_35: 0.9610 - val_loss: 0.2283 - val_auc_35: 0.9621
Epoch 2/500
51/51 [==============================] - 0s 7ms/step - loss: 0.2315 - auc_35: 0.9610 - val_loss: 0.2283 - val_auc_35: 0.9621
Epoch 3/500
51/51 [==============================] - 0s 6ms/step - loss: 0.2315 - auc_35: 0.9610 - val_loss: 0.2283 - val_auc_35: 0.9621
Epoch 4/500
51/51 [==============================] - 0s 6ms/step - loss: 0.2315 - auc_35: 0.9610 - val_loss: 0.2283 - val_auc_35: 0.9621
Epoch 5/500
51/51 [==============================] - 0s 7ms/step - loss: 0.2315 - auc_35: 0.9610 - val_loss: 0.2283 - val_auc_35: 0.9621
Epoch 6/500
51/51 [==============================] - 0s 6ms/step - loss: 0.2315 - auc_35: 0.9610 - val_loss: 0.2283 - val_auc_35: 0.9621
Epoch 7/500
51/51 [==============================] - 0s 6ms/step - loss: 0.2315 - auc_35: 0.9610 - val_loss: 0.2283 - val_auc_35: 0.9621


## Saving the model


In [178]:
NN_model.save('NN_model2')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
INFO:tensorflow:Assets written to: /content/drive/My Drive/NN_model2/assets


### How to load the model

In [33]:
NN_model_check_2 = tf.keras.models.load_model('Final_Model/NN_model2')

# Model Score

In [34]:
from sklearn import metrics
import numpy as np
y_predicted = NN_model_check_2.predict(x_test)
print("AUC : " + str(metrics.roc_auc_score(y_test,y_predicted)))## This score will give the prediction whether the person is likely to pay or not
print("Accuracy : {0}".format(metrics.accuracy_score(np.argmax(y_test,axis =1),np.argmax(y_predicted,axis =1))))

AUC : 0.9632571948717533
Accuracy : 0.908343958067715


# Predicted Probability of Account Paying Us Back

In [36]:
y_predicted[:,1] 

array([9.9726117e-01, 1.3662697e-05, 1.4617925e-05, ..., 9.9774867e-01,
       9.8058403e-01, 9.9998701e-01], dtype=float32)

# Actual class of account indicating whether the User paid us or not

In [38]:
y_test[:,1]

array([1., 0., 0., ..., 1., 1., 1.], dtype=float32)

# Extra Models

## NN model 1

In [24]:
NN_model_check = tf.keras.models.load_model('Extra Models/NN_model1')

In [26]:
from sklearn.model_selection import train_test_split
# Splitting Data into Training and Testing Data
x_train,x_val,y_train,y_val = train_test_split(X_scaled,Y,random_state =7,test_size=0.3)

# Splitting Data into Training and Testing Data
x_test,x_val,y_test,y_val = train_test_split(x_val,y_val,random_state =7,test_size=0.5)

In [27]:
from sklearn import metrics

y_predicted = NN_model_check.predict(x_test)
print("AUC : " + str(metrics.roc_auc_score(y_test,y_predicted)))
print("Accuracy : {0}".format(metrics.accuracy_score(y_test,y_predicted > 0.5)))

AUC : 0.9471305659398136
Accuracy : 0.8795863436747414


# XGboost model should be above 1.2.0

In [33]:
!pip install xgboost==1.4.2

     |████████████████████████████████| 166.7 MB 14 kB/s 


In [1]:
import xgboost as xgb
xgb_mod = xgb.XGBClassifier()
xgb_mod.load_model('xgb.json')

In [18]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
# Splitting Data into Training and Testing Data
x_train_tree,x_test,y_train_tree,y_test = train_test_split(X,Y,random_state =7,test_size=0.3)

In [19]:
y_predicted = xgb_mod.predict_proba(x_test)
print("Score of the intialized model " + str(xgb_mod.score(x_test,y_test)))
print("AUC : " + str(metrics.roc_auc_score(y_test,y_predicted[:,1])))

Score of the intialized model 0.8450913727156821
AUC : 0.9172939816525976
